In [1]:
#Inbound Process
import logging
import paramiko
import sys
import shutil
import time
import posixpath
import datetime
import os
import smtplib, getpass, json

#Working with config file
with open("config.json") as f:
    config = json.load(f)

#sftp credentials
sftp_host = config["sftp"]["hostname"]
sftp_user = config["sftp"]["username"]
sftp_pass = config["sftp"]["password"]

#email credentials
email_address = config["email"]["address"]
email_pass = config["email"]["password"]

#path variables
local = os.path.join(os.getcwd(), 'Processing')
remote = '/Files/ScannedDocs/'
archive = os.path.join(local, "Archive")

#Logging
get_date = datetime.datetime.now().strftime("%Y%m%d_%H%M")
logging.basicConfig(level=logging.ERROR, filename=f"/Users/homebase/Desktop/Python/Udemy Course/Milestone Projects/OPUS/Processing/Log/InboundError/{get_date}-errorlog.log", filemode="w", format="%(asctime)s - %(levelname)s - %(message)s")

with open(f"/Users/homebase/Desktop/Python/Udemy Course/Milestone Projects/OPUS/Processing/Log/Inbound/{get_date}", 'a') as f:
    f.write(f"============= {get_date} Processing =============\n")

    try:
        #Working with email
        smtp_object = smtplib.SMTP("smtp.gmail.com", 587)
        smtp_object.ehlo()
        smtp_object.starttls()
        smtp_object.ehlo()
        smtp_object.login(email_address,email_pass)
        from_address = email_address
        to_address = email_address
        subject = "Inbound Reporting"
        messages = []
    except Exception as e:
        f.write("Failed to send email:", e)

    
    try:
        #Paramiko instancw
        client = paramiko.SSHClient()
        client.load_system_host_keys()
        client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        client.connect(hostname = '192.168.64.3', username = 'testuser', password = 'TempPassword1')
        f.write("Connection successful\n")  
        messages.append("SFTP Connection Successful")
        try:
            with client.open_sftp() as sftp:
                for folder, sub_folders, files in os.walk(local):
                    if os.path.basename(folder) in ['Archive', 'Log', 'Outbound']: #Ignore these folders
                        continue
            
                    for file in files:
                        if file.endswith(".pdf") or file.endswith(".csv"):
                            local_source = os.path.join(folder, file)
                            remote_destination = posixpath.join(remote, file)
                            sftp.put(local_source, remote_destination)
                            f.write(f"Uploading {os.path.basename(local_source)} to {remote}\n")
                            messages.append(f"Uploading {os.path.basename(local_source)} to {remote}")
                        if file.endswith(".pdf"):
                            time.sleep(2)
            
                        if file.endswith(".csv"):
                            time.sleep(10)
        except Exception as e:
            logging.exception("An error occurred!: {e}")
            messages.append(f"Error handling {file}")
            sys.exit() #to stop the script if an error occurs
            
    except Exception as e:
        logging.exception(f"Error during SFTP connection: {e}")
        messages.append(f"Error during SFTP connection: {e}")
        sys.exit() #to stop the script if an error occurs
    finally:
        client.close()
    try: 
        for folder in os.listdir(local):
            if os.path.basename(folder) in ['Archive', 'Log', '.ipynb_checkpoints', 'Outbound']: #Ignore these folders
                    continue
            source = os.path.join(local, folder)
            shutil.move(source, archive)
            f.write(f"{os.path.basename(folder)} moved into Archive")
            messages.append(f"{os.path.basename(folder)} moved into Archive")
    except Exception as e:
            logging.exception(f"Error handling {folder}: {e}")
            messages.append(f"Error handling {folder}")
    finally:
        msg_body = "\n".join(messages)
        msg = "Subject: "+subject+'\n'+ msg_body
        smtp_object.sendmail(from_address,to_address,msg)
        smtp_object.quit()

TypeError: TextIOWrapper.write() takes exactly one argument (2 given)